In [1]:
%load_ext autoreload
%autoreload 2

from transformer_lens import HookedTransformer
import torch as t
from huggingface_hub import login
from contextlib import contextmanager
import json
import torch.nn.functional as F
from tqdm.notebook import tqdm
import gc
from IPython.display import display, HTML
import html as html_escaper
import copy

from utils import *

# login("")
t.set_grad_enabled(False)

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
model_base = HookedTransformer.from_pretrained_no_processing("gemma-2-2b", device="cpu")
model_chat = HookedTransformer.from_pretrained_no_processing("gemma-2-2b-it", device="cuda")
model_hs = copy.deepcopy(model_chat)
clear_mem()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


In [9]:
MAX_SEQ_LEN = 300
NUM_LINES_UNFILTERED = 5000
NUM_LINES = 2000
with open("2b_it_generations_deduped.jsonl", "r") as f:
    data = [json.loads(line)['conversation'] for i, line in enumerate(f) if i < NUM_LINES_UNFILTERED]
toks = model_base.to_tokens(data, prepend_bos=False)[:, :MAX_SEQ_LEN]
# Filter out sequences containing padding tokens (token ID 0)
toks = t.stack([seq for seq in toks if t.all(seq != 0)])[:NUM_LINES]

In [12]:
def vis(
        layer, 
        head, 
        type="hs", # options "hs" ,"ablated"
        max_to_show=10, 
        cscale=0.1
        ):
    
    with open(f"results/topk_data/{type}/L{layer}H{head}.json", "r") as f:
        topk_data = json.load(f)

    for i, (batch_idx, seq_idx) in enumerate(topk_data["inds"]):
        if i > max_to_show:
            break
        highlight_kl(model_base, model_hs, model_chat, toks[batch_idx, :seq_idx+10], layer, head, cscale=cscale, ablate=(type=="ablated"))
        print("--"*100)

In [26]:
vis(19,0, type="hs", cscale=0.02, max_to_show=20)

High KL: 0.0408 at token '1'\n


,Token,p_HS,p_B
0,3,0.295,0.434
1,2,0.034,0.031
2,1,0.668,0.531


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0193 at token ' ultimate'\n


,Token,p_HS,p_B
0,purpose,0.535,0.629
1,God,0.068,0.056
2,divine,0.153,0.121
3,deity,0.043,0.030


...

,Token,p_HS,p_B
0,waarom,0.162,0.183
1,de,0.077,0.081
2,ver,0.089,0.093
3,moti,0.017,0.015
4,motive,0.197,0.163
5,uit,0.023,0.019


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


High KL: 0.0060 at token '<start_of_turn>'\n


,Token,p_HS,p_B
0,ⓘ,0.052,0.063
1,info,0.162,0.183
2,ℹ,0.088,0.096
3,,0.412,0.365
4,,0.022,0.019


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
